In [ ]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import logging 
from matplotlib.patches import PathPatch
from matplotlib.path import Path
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING) 
plt.rcParams['figure.figsize'] = [9.5*2, 6*2]


# XCLIM and xarray
import xclim.indices as xci
import xclim.temperature as temp
import xclim.precip as prec
import numpy as np
import xclim.utils as xut
import xarray as xr
import dask
from distributed import Client

# start client
client=Client(n_workers=4, threads_per_worker=10,diagnostics_port=8787, memory_limit='10GB')
client

In [ ]:
inrep = '/pvcs1/DATA/testdata/XCLIM_workshop/'
rcps = ['rcp45','rcp85']
v = 'tx_max'
ncfiles = {}
for r in rcps:
    ncfiles[r] = glob.glob(glob.os.path.join(inrep,r,v,'*.nc'))
print(ncfiles)

In [ ]:
ens_rcp45 = xut.create_ensemble(ncfiles['rcp45'])

ens_rcp85 = xut.create_ensemble(ncfiles['rcp85'])

In [ ]:
print(ens_rcp45,'\n',ens_rcp85)

In [ ]:
# Plot individual simulations
for i in np.arange(0,ens_rcp45[v].shape[0]):
    plt.plot(ens_rcp45[v][i,].mean(['lon','lat']),color='blue',linewidth=0.2)
    plt.plot(ens_rcp85[v][i,].mean(['lon','lat']),color='red',linewidth=0.2)



In [ ]:

ens_stats = xut.ensemble_mean_std_max_min(ens_rcp85)
print(ens_stats.data_vars)

In [ ]:
plt.rcParams['figure.figsize'] = [9.5*2, 6*2]
x = ens_stats['tx_max_mean'].time.dt.year
y1 = ens_stats['tx_max_max'].mean(['lon','lat'])
y2 = ens_stats['tx_max_min'].mean(['lon','lat'])
plt.fill_between(x,y1,y2, facecolor='red', alpha=0.25)
plt.plot(x,ens_stats['tx_max_mean'].mean(['lon','lat']), color='red', linewidth = 1)


In [ ]:
ens_perc = xut.ensemble_percentiles(ens_rcp85)
print(ens_perc.data_vars)
ens_perc = xut.ensemble_percentiles(ens_rcp85, values=[10,25,50,75,90])
print(ens_perc.data_vars)

In [ ]:
plt.rcParams['figure.figsize'] = [9.5*2, 6*2]
y1 = ens_perc['tx_max_p90'].mean(['lon','lat'])
y2 = ens_perc['tx_max_p10'].mean(['lon','lat'])
plt.fill_between(x,y1,y2, facecolor='red', alpha=0.15)

y1 = ens_perc['tx_max_p75'].mean(['lon','lat'])
y2 = ens_perc['tx_max_p25'].mean(['lon','lat'])
plt.fill_between(x,y1,y2, facecolor='red', alpha=0.2)
plt.plot(x,ens_perc['tx_max_p50'].mean(['lon','lat']), color='red', linewidth = 1)


In [ ]:
# 30 year averages
ensSub = xut.subset_bbox(ens_rcp85,start_yr = 1951)
ens30y = ensSub.rolling(time=30,  center=True).mean()

In [ ]:
#print(ens30y)
print(ens30y.sel(time='1980-01-01'))
ens30y.sel(time='1980-01-01')['tx_max'][0,:,:].plot.imshow()
#print(ens30y.time)

In [ ]:
# Create deltas wrt to 1981-2010
ref_per = ens30y.sel(time='2010-01-01')
print(ref_per.time.values)

In [ ]:
ensDeltas = ens30y - ref_per
print(ensDeltas)

In [ ]:
ens_percDeltas = xut.ensemble_percentiles(ensDeltas)
print(ens_percDeltas)

In [ ]:
ens_percDeltas['tx_max_p50'].isel(time=-1).plot.imshow()